<a href="https://colab.research.google.com/github/alptugo/me536_final_countdown/blob/main/ME536_Final_Countdown_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting the drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Randomly separating 3 of the 14 lattice types into "not learned" group. Getting label array as well for trainning.

In [ ]:
from PIL import Image
import numpy as np
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from random import sample
import pandas as pd
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, confusion_matrix
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Conv2DTranspose, UpSampling2D, Dropout
from tensorflow.keras.metrics import binary_crossentropy
import pickle

#Giving the directory of the folder by folder 4 channel images
path = "/content/drive/MyDrive/me536_final_countdown/sep_multi_channel_images/"
lattice_types  = os.listdir(path)

#defining label to number mapping dictionary
label_map = {label:num for num, label in enumerate(lattice_types)}

#getting random lattice types to exclude from trainning
excluded_lattice_types=[lattice_types[i] for i in sample(range(14), 3)]

print(f"I excluded {excluded_lattice_types}")

#for the first 200 images of every lattice types, opens them, transforms their
#channels to binary (from 0 or 255 to 0 or 1). Gets the images and labels to different
#lists depending on wether they are excluded or not
sequences, excluded_sequences, labels, excluded_labels = [], [], [], []
for lattice_type in lattice_types:
    for sequence in range(200):
      img = Image.open("{}{}/{} ({}).png".format(path, lattice_type, lattice_type, sequence+1))
      img_array = np.array(img)
      img_array = img_array / 255.0
      if lattice_type not in excluded_lattice_types:
          sequences.append(img_array)
          labels.append(label_map[lattice_type])
      else:
          excluded_sequences.append(img_array)
          excluded_labels.append(label_map[lattice_type])

I excluded ['Truncated octahedron', 'Truncated cube', 'Simple cubic']


In [ ]:
label_map

{'Body centered cubic': 0,
 'Column': 1,
 'Diamond': 2,
 'Re-entrant': 3,
 'Kelvin cell': 4,
 'IsoTruss': 5,
 'Columns': 6,
 'Octet': 7,
 'Face centered cubic': 8,
 'Fluorite': 9,
 'Simple cubic': 10,
 'Truncated octahedron': 11,
 'Truncated cube': 12,
 'Weaire-Phelan': 13}

Creating new label_map by jumping excluded lattice types.

In [ ]:
new_label_map={label:num for num, label in enumerate([lattice_type for lattice_type in lattice_types if lattice_type not in excluded_lattice_types])}


Mapping from old label map new one

In [ ]:
old_to_new_label_map={}
for key in label_map.keys():
  if key in new_label_map.keys():
    old_to_new_label_map = {**old_to_new_label_map, label_map[key]: new_label_map[key]}
old_to_new_label_map

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 13: 10}

Updating the labels with label map

In [ ]:
labels=[old_to_new_label_map[label] for label in labels]

Transforming the labels into keras type structure (there is an array consisted of all classes and where the 1 value represents inclusion on that class).

In [ ]:
y = tf.keras.utils.to_categorical(labels).astype(int)
y.shape

(2200, 11)

Transforming the images into a single numpy array. Splitting this array into lattice types for autoencoder trainning.
Also splitting this array and corresponding labels into test and train parts for classifier trainning.

In [ ]:
X = np.array(sequences)
print(X.shape)

X_by_examples=np.array_split(X, 11)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, shuffle=True, stratify=y)

(2200, 48, 48, 4)


In [ ]:
print(X_train.shape, y_train.shape)

(1980, 48, 48, 4) (1980, 11)


Defining a CNN model.
The convolutional layers extract features from the input images, while the dense layers perform classification based on those features. The max pooling layers and dropout layer are used to reduce the dimensionality and prevent overfitting.

In [ ]:
image_input = tf.keras.Input(shape=(48, 48, 4), name='image_input')

x = image_input
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(11, activation='softmax')(x)

lattice_classifier = tf.keras.Model(inputs=image_input, outputs=output)

Defining an optimizer for the model. Followed by compiling and trainning.

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.002, beta_1=0.9, beta_2=0.9999, epsilon=1e-8, amsgrad=False)
lattice_classifier.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
lattice_classifier.fit(X_train, y_train, epochs=8, batch_size=32, validation_split=0.1, shuffle=True)


test_loss, test_acc = lattice_classifier.evaluate(X_test, y_test)
print('Model Summary:', lattice_classifier.summary())

Epoch 1/8
56/56 [==============================] - 7s 112ms/step - loss: 0.6931 - categorical_accuracy: 0.7677 - val_loss: 0.0911 - val_categorical_accuracy: 0.9848
Epoch 2/8
56/56 [==============================] - 6s 109ms/step - loss: 0.0592 - categorical_accuracy: 0.9837 - val_loss: 0.0324 - val_categorical_accuracy: 0.9798
Epoch 3/8
56/56 [==============================] - 6s 109ms/step - loss: 0.0323 - categorical_accuracy: 0.9927 - val_loss: 0.0071 - val_categorical_accuracy: 1.0000
Epoch 4/8
56/56 [==============================] - 6s 110ms/step - loss: 0.0472 - categorical_accuracy: 0.9860 - val_loss: 0.0051 - val_categorical_accuracy: 1.0000
Epoch 5/8
56/56 [==============================] - 6s 111ms/step - loss: 0.0193 - categorical_accuracy: 0.9949 - val_loss: 0.0087 - val_categorical_accuracy: 0.9949
Epoch 6/8
56/56 [==============================] - 6s 110ms/step - loss: 0.0075 - categorical_accuracy: 0.9989 - val_loss: 0.0068 - val_categorical_accuracy: 0.9949
Epoch 7/8


Checking the models perfomence with Confusion Matrix. Some different displays are commented out as they do not fit screen.

In [ ]:
yhat = lattice_classifier.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

row_labels = [lattice_type for lattice_type in lattice_types if lattice_type not in excluded_lattice_types]
column_labels = [lattice_type for lattice_type in lattice_types if lattice_type not in excluded_lattice_types]
df = pd.DataFrame(confusion_matrix(ytrue, yhat), columns=column_labels, index=row_labels)

# print('Confusion Matrix:\n', df, '\n')
# print('Accuracy Score:', accuracy_score(ytrue, yhat), '\n')
print('Confusion Matrix:\n', confusion_matrix(ytrue, yhat), '\n')
# print('Multilabel Confusion Matrix:\n', multilabel_confusion_matrix(ytrue, yhat))

7/7 [==============================] - 0s 30ms/step
Confusion Matrix:
 [[20  0  0  0  0  0  0  0  0  0  0]
 [ 0 19  0  0  0  0  0  0  0  1  0]
 [ 0  0 20  0  0  0  0  0  0  0  0]
 [ 0  0  0 20  0  0  0  0  0  0  0]
 [ 0  0  0  0 20  0  0  0  0  0  0]
 [ 0  0  0  0  0 20  0  0  0  0  0]
 [ 0  0  0  0  0  0 20  0  0  0  0]
 [ 0  0  0  0  0  0  0 20  0  0  0]
 [ 0  0  0  0  0  0  0  0 20  0  0]
 [ 0  0  0  0  0  0  0  0  0 20  0]
 [ 0  0  0  0  0  0  0  0  0  0 20]] 



Defining the first 20 element of each learned lattice type as autoencoder test data. Rest are used for trainning.

In [ ]:
X_by_examples_train, X_by_examples_test=[X_by_example[20:] for X_by_example in X_by_examples], [X_by_example[:20] for X_by_example in X_by_examples]

For every type of learned lattice, a new autoencoder is defined and trained. Each autoencoder generated is appended to a list. The same is performed for encoder part as well, for a possible future need.

The convolutional layers with kernel size of (3,3) and relu activation function extract features from the input image.

The max pooling layers reduce the spatial dimension of the feature maps by taking the maximum value of a defined neighborhood.

The Dropout layers are used to prevent overfitting by randomly setting a fraction of input units to 0.

The encoded layer represent the lower dimensional representation of the input image.

The UpSampling layers increase the spatial dimensions of the feature maps.

The final decoded layer reconstructs the input image by using sigmoid activation function.

In [ ]:
autoencoders, encoders=[], []

#for every lattice type that is not excluded
for i in range(len(X_by_examples_train)):

  # Create the input layer
  input_img = Input(shape=(48, 48, 4))

  # Encoder layers
  x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
  x = Dropout(0.1)(x)
  x = MaxPooling2D((2, 2), padding='same')(x)
  x = Dropout(0.1)(x)
  x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
  x = Dropout(0.1)(x)
  x = MaxPooling2D((2, 2), padding='same')(x)

  encoded = Conv2D(8, (3, 3), activation='relu', padding='same', name="encoded")(x)

  # Decoder layers
  x = UpSampling2D((2, 2))(encoded)
  x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
  x = UpSampling2D((2, 2))(x)
  x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
  decoded = Conv2D(4, (3, 3), activation='sigmoid', padding='same')(x)


  # Creates the autoencoder model
  autoencoder = Model(input_img, decoded)
  autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

  # Trains the autoencoder on training data
  autoencoder.fit(X_by_examples_train[i], X_by_examples_train[i], epochs=10, batch_size=10, validation_split=0.1)
  autoencoders.append(autoencoder)

  encoder = Model(input_img, encoded)
  encoders.append(encoder)

Epoch 1/10
17/17 [==============================] - 3s 107ms/step - loss: 0.6857 - val_loss: 0.6579
Epoch 2/10
17/17 [==============================] - 1s 71ms/step - loss: 0.6365 - val_loss: 0.5939
Epoch 3/10
17/17 [==============================] - 1s 70ms/step - loss: 0.5661 - val_loss: 0.4619
Epoch 4/10
17/17 [==============================] - 1s 73ms/step - loss: 0.4381 - val_loss: 0.2926
Epoch 5/10
17/17 [==============================] - 2s 115ms/step - loss: 0.3357 - val_loss: 0.2180
Epoch 6/10
17/17 [==============================] - 2s 108ms/step - loss: 0.2771 - val_loss: 0.1787
Epoch 7/10
17/17 [==============================] - 1s 73ms/step - loss: 0.2380 - val_loss: 0.1689
Epoch 8/10
17/17 [==============================] - 1s 71ms/step - loss: 0.2177 - val_loss: 0.1447
Epoch 9/10
17/17 [==============================] - 1s 72ms/step - loss: 0.1987 - val_loss: 0.1292
Epoch 10/10
17/17 [==============================] - 1s 70ms/step - loss: 0.1845 - val_loss: 0.1143
Epoch 

Each encoder is ran through its trainning data, to get a mean reconstruction error error for each lattice class with its encoder. Also the standard deviation is calcualted as well yet it is not used as simple thresholding was found sufficient.

In [ ]:
means, stds, rec_errs=[], [], []

for autoencoder, image_set in zip(autoencoders, X_by_examples_train):

  #reconstructing data
  autoencoded_imgs = autoencoder.predict(image_set)

  #getting the simple difference mean as well as binary crossentropy error means
  mean = np.mean(np.power(image_set - autoencoded_imgs, 2), axis=0)
  reconstruction_error = binary_crossentropy(image_set, autoencoded_imgs)
  std = np.std(mean)

  rec_errs.append(reconstruction_error)
  means.append(mean)
  stds.append(std)

6/6 [==============================] - 0s 49ms/step


Getting the total of reconstruction error means into a list, so that we would have a single value for every class.

In [ ]:
rec_list=[np.mean(rec_errs[i], axis=0).sum() for i in range(11)]
rec_list

[360.66553,
 32.40017,
 675.1433,
 306.97955,
 532.20056,
 742.09607,
 19.461943,
 452.1493,
 354.76053,
 560.6339,
 540.4055]

Checking if the threshold is valid for test data. If "is_same=True", the img selected and the autoencoder is of the same class.The number of inlier must be 20 (and it is). 

If "is_same=False", it checks for different autoencoder and image groups. The number of inliers must be 0 (and it is almost always).

In [ ]:
#randomly selecting 2 groups to crosscheck encoders with different images
selection_group, image_from_group = sample(range(11), 2)
is_same=False

if is_same:
  selection_group, image_from_group = selection_group, selection_group


print(selection_group, image_from_group)


in_number, out_number=0,0
for in_group_index in range(len(X_by_examples_test[selection_group])):

      #gets the image from the test group
      new_img=X_by_examples_test[image_from_group][in_group_index]
      new_img= np.expand_dims(new_img, axis=0)

      #gets the reconstructed image
      autoencoded_new_img = autoencoders[selection_group].predict(new_img, verbose=0)

      #calculates the reconstruction error sum
      reconstruction_error = binary_crossentropy(new_img, autoencoded_new_img)
      rec_error=np.sum(reconstruction_error)

      #thresholds the reconstruction error with the groups error
      if rec_error>rec_list[selection_group]*2.4:
        out_number+=1
      else:
        in_number+=1

print(in_number)

8 2
0


Getting the inverse of the old and new label map, so that when the model gives a number, we can get the name of the lattice

In [ ]:
inv_map = {label_map[k] : k for k in label_map}

new_inv_map={new_label_map[k] : k for k in new_label_map}

Function that controls if an image is realy from that class. The functionized and singular form the of the previous block.

In [ ]:
def loyalty_checker(autoencoder_list, img, preverdict, rec_list):
  '''This is a function for detecting if a classification is correct or if it
  is the overconfidence of the classifier. The inputs are the autoencder list containing
  the autoencoder for each class, the image being tested, the classification of the classifier
  and the mean reconstruction error sum list conatinaing data for each lattice type.'''

  autoencoded_new_img = autoencoder_list[preverdict].predict(img, verbose=0)
  reconstruction_error = binary_crossentropy(img, autoencoded_new_img)
  rec_error=np.sum(reconstruction_error)

  if rec_error>rec_list[preverdict]*2.4:
    return False
  else:
    return True

In [ ]:
excluded_lattice_types

['Truncated octahedron', 'Truncated cube', 'Simple cubic']

In [ ]:
new_inv_map

{0: 'Body centered cubic',
 1: 'Column',
 2: 'Diamond',
 3: 'Re-entrant',
 4: 'Kelvin cell',
 5: 'IsoTruss',
 6: 'Columns',
 7: 'Octet',
 8: 'Face centered cubic',
 9: 'Fluorite',
 10: 'Weaire-Phelan'}

Loading the dictionary that maps zip file names to 4 channels images, so that the input can be given as both .zip file or .png file.

In [ ]:
with open('/content/drive/MyDrive/me536_final_countdown/file_name_mapper.pickle', 'rb') as handle:
    zip2img_map = pickle.load(handle)

##THIS IS THE RESULT PART!

On this part, it asks for either a .zip or a 4 channel .png file. If a .zip is given, it searches for corresponding image file, if there is no such image file, that means the given .zip file does not represents a 3D mesh file with a lattice. Therefore it gives the output by stating this type of novelty.

If the .zip file corresponds to a 4 channel image, the image is classified. If the lattice is not from the known classes, the novelty is specified by stating that the lattice is from a new class.

If a .png type of data is given, it is guaranteed that the data contains lattices, therefore the output is either a classification or the notification of a novel type of lattice.

In [ ]:
input_path=input("Please give the path of a 4 channel image or a zip. For 4 channel images,\
any index greater than 200 was not used on either training or testing of models,\
so feel free to use them. Enter 'q' to stop.\n")

#As I tried transfer learning earlier, I was changing the model I used when 
#I encountered new data. This things here are relics of that.
last_model=lattice_classifier
new_lattice_types=[]
novelty_counter=0
while input_path!="q":

  #checks the inputs type
  if input_path[-4:]==".png":
    new_img=Image.open(input_path)
  elif input_path[-4:]==".zip":
    zip_name=input_path.split("/")[-1]

    #checks wether the input is transformed to a 4 channel img
    if zip_name in zip2img_map.keys():
      img_name=zip2img_map[zip_name]
      folder_name=zip_name.lstrip('1234567890')[:-4]

      #sets the path to the corresponding image file
      input_path=f"/content/drive/MyDrive/me536_final_countdown/sep_multi_channel_images/{folder_name}/{img_name}"
      print(f"4 channel img found on /content/drive/MyDrive/me536_final_countdown/sep_multi_channel_images/{folder_name}/{img_name}")
      new_img=Image.open(input_path)


    else:
      print("There is no such 4 channel image, your zip doesn't contain lattices...\n")
      input_path=input("Please give the path of a 4 channel image or a zip. For 4 channel images,\
      any index greater than 200 was not used on either training or testing of models,\
      so feel free to use them. Enter 'q' to stop.\n")
      continue
  new_img = np.array(new_img)
  new_img=new_img/255
  new_img= np.expand_dims(new_img, axis=0)

  #gets lattice type as number by predicting through classifier
  type_number=int(np.argmax(last_model.predict(new_img)))

  #gets lattice type name from label map
  type_text=new_inv_map[type_number]
  print(f"Preliminarily it is {type_text},\
  but gotta check if it is an overconfidence against a novelty")
  if type_text not in new_lattice_types:
    #checks wether it is realy from that class or a novelty using loyalty_checker
    #function
    is_inlier=loyalty_checker(autoencoders, new_img, type_number, rec_list)

    if is_inlier:
      print("It really is an inlier\n")
    else:
      print("It is not really from this group. We have an alien\n")


  input_path=input("Please give the path of a 4 channel image or a zip. For 4 channel images,\
  any index greater than 200 was not used on either training or testing of models,\
   so feel free to use them. Enter 'q' to stop.\n")


Please give the path of a 4 channel image or a zip. For 4 channel images,any index greater than 200 was not used on either training or testing of models,so feel free to use them. Enter 'q' to stop.
/content/drive/MyDrive/me536_final_countdown/images/10Simple cubic.zip
There is no such 4 channel image, your zip doesn't contain lattices...

Please give the path of a 4 channel image or a zip. For 4 channel images,      any index greater than 200 was not used on either training or testing of models,      so feel free to use them. Enter 'q' to stop.
/content/drive/MyDrive/me536_final_countdown/images/20Simple cubic.zip
4 channel img found on /content/drive/MyDrive/me536_final_countdown/sep_multi_channel_images/Simple cubic/Simple cubic (59).png
1/1 [==============================] - 0s 19ms/step
Preliminarily it is Re-entrant,  but gotta check if it is an overconfidence against a novelty
It is not really from this group. We have an alien

Please give the path of a 4 channel image or a zip. 

The code below is an attempt on transfer learning, however it only worked correctly once (off many many tries). As the new data set used for adapting is too small (3 images) it doesn't realy function well for now. 

I simply tried to freeze all layers except the last one. I redefined the last layer for new amount of classes, and fed in new data. Moreover I tried to both use 3 images of every class and only the new class.

In [ ]:
inv_map = {label_map[k] : k for k in label_map}

X_train_tiny=np.concatenate([latt_type[:3] for latt_type in X_by_examples], axis=0)

y_train_tiny=[i for i in range(11) for k in range(3)]

y_train_tiny = tf.keras.utils.to_categorical(y_train_tiny).astype(int)

input_path=input("Please give the path of an image. Any index greater than 200\
was not used on either training or testing of models, so feel free to use them.\
Enter 'q' to stop.\n")

last_model=lattice_classifier
new_lattice_types=[]
novelty_counter=0
while input_path!="q":
  if input_path[:-4]==".png":
    new_img=Image.open(input_path)
  elif input_path[:-4]==".zip":
    zip_name=input_path.split("/")[-1]
    if zip_name in zip2img_map.keys():
      img_name=zip2img_map[zip_name]
      folder_name=zip_name.lstrip('1234567890')[:-4]
      input_path=f"/content/drive/MyDrive/me536_final_countdown/sep_multi_channel_images/{folder_name}/{img_name}"
      print(f"4 channel img found on /content/drive/MyDrive/me536_final_countdown/sep_multi_channel_images/{folder_name}/{img_name}")
      new_img=Image.open(input_path)
    else:
      print("There is no such 4 channel image, your zip doesn't contain lattices...")
      continue
  new_img = np.array(new_img)
  new_img=new_img/255

  new_img= np.expand_dims(new_img, axis=0)
  type_number=int(np.argmax(last_model.predict(new_img)))
  type_text=new_inv_map[type_number]
  print(f"Preliminarily it is {type_text},\
  but gotta check if it is an overconfidence against a novelty")
  if type_text not in new_lattice_types:
    is_inlier=loyalty_checker(autoencoders, new_img, type_number, rec_list)

    if is_inlier:
      print("It really is an inlier")
    else:
      print("It is not really from this group. Could you give 2 more of this, so it can learn?")
      input_path=input()

      if input_path[:-4]==".png":
        new_img=Image.open(input_path)
      elif input_path[:-4]==".zip":
        zip_name=input_path.split("/")[-1]
      if zip_name in zip2img_map.keys():
        img_name=zip2img_map[zip_name]
        folder_name=zip_name.lstrip('1234567890')[:-4]
        input_path=f"/content/drive/MyDrive/me536_final_countdown/sep_multi_channel_images/{folder_name}/{img_name}"
        print(f"4 channel img found on /content/drive/MyDrive/me536_final_countdown/sep_multi_channel_images/{folder_name}/{img_name}")
      new_img2=Image.open(input_path)

      new_img2=Image.open(new_img2)
      new_img2 = np.array(new_img2)
      new_img2=new_img/255


      if input_path[:-4]==".png":
        new_img=Image.open(input_path)
      elif input_path[:-4]==".zip":
        zip_name=input_path.split("/")[-1]
      if zip_name in zip2img_map.keys():
        img_name=zip2img_map[zip_name]
        folder_name=zip_name.lstrip('1234567890')[:-4]
        input_path=f"/content/drive/MyDrive/me536_final_countdown/sep_multi_channel_images/{folder_name}/{img_name}"
        print(f"4 channel img found on /content/drive/MyDrive/me536_final_countdown/sep_multi_channel_images/{folder_name}/{img_name}")
      new_img3=Image.open(input_path)

      new_img3=input()
      new_img3=Image.open(new_img3)
      new_img3 = np.array(new_img3)
      new_img3=new_img/255


      last_layer_of_classifier=last_model.layers[-2]
      new_class_count=last_model.layers[-1].output_shape[1]+1
      new_dense_layer=tf.keras.layers.Dense(new_class_count, activation='softmax')(last_layer_of_classifier.output)
      new_lattice_classifier=Model(inputs=last_model.input, outputs=new_dense_layer)

      for layer in lattice_classifier.layers[:-1]:
        layer.trainable = False
      # X_train_new=np.concatenate((X_train, new_img, new_img2, new_img3), axis=0)

      if novelty_counter==0:

        # y_train_tiny=np.hstack((y_train_tiny, np.zeros((33,1))))

        novelty_label_train=np.zeros((3, new_class_count))
        novelty_label_train[:,[-1]]=np.ones((3,1))
        novelty_label_train=novelty_label_train.astype(int)
        # novelty_label_train=np.concatenate((y_train_tiny, novelty_label_train), axis=0)


        # novelty_train=np.concatenate((X_train_tiny, new_img, new_img2, new_img3), axis=0)
        novelty_train=np.concatenate((new_img, new_img2, new_img3), axis=0)


      else:
        new_rows=np.zeros((3, new_class_count))
        new_rows[:,[-1]]=np.ones((3,1))
        new_rows=new_rows.astype(int)
        novelty_label_train=np.hstack((novelty_label_train, np.zeros((novelty_label_train.shape[0],1))))
        novelty_label_train=np.concatenate((novelty_label_train, new_rows), axis=0)

        novelty_train=np.concatenate((novelty_train, new_img, new_img2, new_img3), axis=0)

      opt = tf.keras.optimizers.Adam(learning_rate=0.002, beta_1=0.9, beta_2=0.9999, epsilon=1e-8, amsgrad=False)
      new_lattice_classifier.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
      
      early_stop = tf.keras.callbacks.EarlyStopping(monitor='categorical_accuracy', min_delta=0.001, patience=5, mode='max', restore_best_weights=True)

      new_lattice_classifier.fit(novelty_train, novelty_label_train, epochs=30, batch_size=3, validation_split=0.4, shuffle=True, callbacks=[early_stop])
      
      new_label_map[f"NewLattice{novelty_counter}"]=new_class_count-1
      new_inv_map[new_class_count-1]=f"NewLattice{novelty_counter}"
      new_lattice_types.append(f"NewLattice{novelty_counter}")


      print(f"New lattice type called NewLattice{novelty_counter} is defined. You can give it a try!")
      last_model=new_lattice_classifier
      novelty_counter+=1
  else:
    print("Well it seems like this type is learned on the fly, so there isn't\
     enough data to crosscheck this classification.")

  input_path=input("Please give the path of an image. Any index greater than 200\
  was not used on either training or testing of models, so feel free to use them.\
  Enter 'q' to stop.\n")
